In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

LOAD MODELS

loading the Recon3D model will give an error because the compartments have not been expressed in the right format for COBRApy

In [2]:
# load RPE, PR, Recon3D, and basis Human1 models from cs mods folder
from pathlib import Path
from cobra.io import read_sbml_model, load_matlab_model
mod = read_sbml_model(Path().cwd() / 'models' / 'Human-GEM.xml' )
mod_RPE = read_sbml_model(Path().cwd() / 'models/cs_mods' / 'model_06_MeanExpression_RPE__VoigtEtAl2019_.xml')
mod_PR = read_sbml_model(Path().cwd() / 'models/cs_mods' / 'model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml' )
mod_Recon3D = load_matlab_model(Path().cwd() / 'models' / 'Recon3D_model2_corrected_subsystems.mat' )

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


No defined compartments in model model2. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


MODIFY MODELS

add back lost exchange reactions and ATP demand reaction.

In [3]:
# modify context specific models
from src.modify_model import add_all_EX_rxns, remove_compartment

for m in [mod_RPE, mod_PR]: # add ATP DM rxn and lost EX rxns to context specific models
    m.add_reactions([mod.reactions.get_by_id('MAR03964').copy()]) # make sure it contains ATP DM rxn
    m = add_all_EX_rxns(mod,m) # add lost EX rxns
    for r in [r for r in m.reactions if len(r.products)==0]: 
        r.bounds = (0,1000) # close lb for exchange reactions

Remove nucleus from PR models (and control equivalent)

In [4]:
mod_PR = remove_compartment(mod_PR.copy(), 'n')  # remove nucleus ('n') from PR model
mod_no_nucleus = remove_compartment(mod.copy(), 'n')  # remove nucleus ('n') from Human1 model
mod_Recon3D_no_nucleus = remove_compartment(mod_Recon3D.copy(), 'n')  # remove nucleus ('n') from Human1 model

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpeot_89jx.lp
Reading time = 0.02 seconds
: 5013 rows, 12490 columns, 46478 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpfm_aftf1.lp
Reading time = 0.04 seconds
: 8369 rows, 26140 columns, 111668 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpn01096jz.lp
Reading time = 0.04 seconds
: 5835 rows, 21200 columns, 80850 nonzeros


In [ ]:
    from src.modify_model import fix_compartment_dict, add_rxns2groups, add_id_suffix
    from src.combine_RPE_PR import add_interface_RPE, add_interface_PR, fuse_RPE_PR, del_RPE_PR_dupl_rxns
    mod_PR = add_id_suffix(mod_PR, '_PR')
    mod_RPE = add_id_suffix(mod_RPE, '_RPE')
    mod_RPE = add_interface_RPE(mod_RPE, 'e_RPE')
    mod_PR = add_interface_PR(mod_PR, 'e_PR')
    mod_RPE_PR = fuse_RPE_PR(mod_RPE,mod_PR)

In [ ]:
    mod_RPE_PR = fix_compartment_dict(mod_RPE_PR)
    mod_RPE_PR = add_rxns2groups(mod_RPE_PR)
    mod_RPE_PR = del_RPE_PR_dupl_rxns(mod_RPE_PR)

In [ ]:
# make combined RPE_PR models
from src.combine_RPE_PR import make_RPE_PR_model, make_RPE_PR_model_Recon3D
mod_RPE_PR = make_RPE_PR_model(mod_RPE.copy(), mod_PR.copy())
mod_Human1_Human1 = make_RPE_PR_model(mod.copy(), mod_no_nucleus.copy())
mod_Recon3D_Recon3D = make_RPE_PR_model_Recon3D(mod_Recon3D.copy(), mod_Recon3D_no_nucleus.copy())

In [ ]:
from cobra.io import write_sbml_model
# save combined RPE_PR models
write_sbml_model(mod_RPE_PR, "mod_RPE_PR.xml")
write_sbml_model(mod_Human1_Human1, "mod_Human1_Human1.xml")
write_sbml_model(mod_Recon3D_Recon3D, "mod_Recon3D_Recon3D.xml")